In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
from sklearn.impute import SimpleImputer
from deep_translator import GoogleTranslator
import re
from bs4 import BeautifulSoup


In [2]:
readers = pd.read_csv('C:\\Users\\a814810\\OneDrive - Atos\\Documents\\readers.csv')

In [3]:
readers.head()

,id,art_id
0,1,ld.154103
1,1,ld.142559
2,1,1.18331199
3,1,ld.144819
4,1,ld.1293110


In [4]:
readers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27855 entries, 0 to 27854
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      27855 non-null  int64 
 1   art_id  27855 non-null  object
dtypes: int64(1), object(1)
memory usage: 435.4+ KB


In [5]:
articles = pd.read_json('C:\\Users\\a814810\\OneDrive - Atos\\Documents\\articles.json', encoding="utf-8", lines=True)
articles.head()

,nzz_id,author,catchline,content,content_length,department,lead_text,pub_date,title,flag
0,ld.149648,Claudia Gabriel,Obligationenfonds mit fixer Laufzeit,"<div data-component=""subtitle"" class=""subtitle...",704,Finanzen,Die Idee ist gut: Statt einer einzigen Obligat...,2017-03-09 08:01:21.0,Es gibt noch interessante Varianten,NaN
1,1.18145900,NaN,NaN,,0,Panorama,Zum Auftakt der Fashion Week in New York zeige...,2017-04-11 14:00:29.473,Fashion Week New York,
2,ld.138769,NaN,E-Banking-Ausfall,"<div data-component=""subtitle"" class=""subtitle...",523,Finanzen,Seit Sonntag funktioniert das E-Banking der Po...,2017-01-09 13:55:00.0,Postfinance kämpft mit dem System,NaN
3,ld.143700,NaN,Terror in Frankreich,"<div data-component=""subtitle"" class=""subtitle...",180,International,Einen Tag nach dem Angriff auf Soldaten beim P...,2017-02-04 12:50:25.0,Louvre nach Macheten-Angriff wieder geöffnet,NaN
4,ld.149385,NaN,Unglück in Panama,"<div data-component=""subtitle"" class=""subtitle...",117,Panorama,Bei einem Busunglück in Panama sind 17 Persone...,2017-03-06 07:31:21.0,Bus prallt gegen eine Mauer und stürzt in Fluss,NaN


In [6]:
articles.shape

(22025, 10)

In [7]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22025 entries, 0 to 22024
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   nzz_id          22025 non-null  object
 1   author          11453 non-null  object
 2   catchline       17872 non-null  object
 3   content         21950 non-null  object
 4   content_length  22025 non-null  int64 
 5   department      20389 non-null  object
 6   lead_text       20410 non-null  object
 7   pub_date        22025 non-null  object
 8   title           22019 non-null  object
 9   flag            6675 non-null   object
dtypes: int64(1), object(9)
memory usage: 1.7+ MB


In [8]:
# missing values
articles = articles.replace("", np.nan)

In [9]:
total = articles.isna().sum().sort_values(ascending=False)
percent = (articles.isna().sum()/articles.isna().count()).round(4).sort_values(ascending=False)
missing_data = pd.concat([total,percent], axis=1, keys=['Total','Percent'])
missing_data

,Total,Percent
flag,19125,0.8683
author,10802,0.4904
content,7120,0.3233
catchline,4154,0.1886
department,3241,0.1472
lead_text,2334,0.1060
title,6,0.0003
nzz_id,0,0.0000
content_length,0,0.0000
pub_date,0,0.0000


In [10]:
# usunięcie niepotrzebnych kolumn
articles = articles.drop(['flag'], axis=1)

In [11]:
# zmiana typu
articles.pub_date = pd.to_datetime(articles["pub_date"])

In [12]:
articles.head()

,nzz_id,author,catchline,content,content_length,department,lead_text,pub_date,title
0,ld.149648,Claudia Gabriel,Obligationenfonds mit fixer Laufzeit,"<div data-component=""subtitle"" class=""subtitle...",704,Finanzen,Die Idee ist gut: Statt einer einzigen Obligat...,2017-03-09 08:01:21.000,Es gibt noch interessante Varianten
1,1.18145900,NaN,NaN,NaN,0,Panorama,Zum Auftakt der Fashion Week in New York zeige...,2017-04-11 14:00:29.473,Fashion Week New York
2,ld.138769,NaN,E-Banking-Ausfall,"<div data-component=""subtitle"" class=""subtitle...",523,Finanzen,Seit Sonntag funktioniert das E-Banking der Po...,2017-01-09 13:55:00.000,Postfinance kämpft mit dem System
3,ld.143700,NaN,Terror in Frankreich,"<div data-component=""subtitle"" class=""subtitle...",180,International,Einen Tag nach dem Angriff auf Soldaten beim P...,2017-02-04 12:50:25.000,Louvre nach Macheten-Angriff wieder geöffnet
4,ld.149385,NaN,Unglück in Panama,"<div data-component=""subtitle"" class=""subtitle...",117,Panorama,Bei einem Busunglück in Panama sind 17 Persone...,2017-03-06 07:31:21.000,Bus prallt gegen eine Mauer und stürzt in Fluss


In [13]:
articles.shape

(22025, 9)

In [14]:
# uzupełnianie/usuwanie braków
# title 
articles.query('title.isna()') 

,nzz_id,author,catchline,content,content_length,department,lead_text,pub_date,title
3385,ld.1287664,NaN,Vor 50 Jahren,NaN,1,Panorama,Eine Frau an einem Marathon? Vor 50 Jahren noc...,2017-04-19 12:45:25.280,NaN
3764,ld.1289145,Alan Niederer,Astronomie,"<header class=""headline""><div class=""headline_...",20,Wissenschaft,NaN,2017-04-27 12:42:15.251,NaN
7440,ld.154164,Susanna Rusterholz,Bundesratssitzung extra muros,NaN,0,Schweiz,Zum zwölften Mal seit 2010 trifft sich der Bun...,2017-03-29 13:07:41.963,NaN
12943,ld.1085897,Andi Maerz,NaN,"<header class=""headline""><div class=""headline_...",125,International,NaN,2017-04-12 13:49:45.074,NaN
14855,ld.1086020,NaN,Donald Trump,"<header class=""headline""><div class=""headline_...",297,Sport,NaN,2017-04-13 06:36:28.852,NaN
19178,ld.1294531,Andrea Hutter,NaN,"<header class=""headline""><div class=""headline_...",8,NaN,NaN,2017-05-18 10:19:08.107,NaN


In [15]:
articles = articles[articles.title.notna()]

In [16]:
colnames = articles.columns # zachowanie nazw kolumn

In [17]:
imputer  = SimpleImputer(strategy='constant',fill_value='Unknown')
articles = pd.DataFrame(imputer.fit_transform(articles))
articles.columns = colnames

In [18]:
articles.head()

,nzz_id,author,catchline,content,content_length,department,lead_text,pub_date,title
0,ld.149648,Claudia Gabriel,Obligationenfonds mit fixer Laufzeit,"<div data-component=""subtitle"" class=""subtitle...",704,Finanzen,Die Idee ist gut: Statt einer einzigen Obligat...,2017-03-09 08:01:21.000,Es gibt noch interessante Varianten
1,1.18145900,Unknown,Unknown,Unknown,0,Panorama,Zum Auftakt der Fashion Week in New York zeige...,2017-04-11 14:00:29.473,Fashion Week New York
2,ld.138769,Unknown,E-Banking-Ausfall,"<div data-component=""subtitle"" class=""subtitle...",523,Finanzen,Seit Sonntag funktioniert das E-Banking der Po...,2017-01-09 13:55:00.000,Postfinance kämpft mit dem System
3,ld.143700,Unknown,Terror in Frankreich,"<div data-component=""subtitle"" class=""subtitle...",180,International,Einen Tag nach dem Angriff auf Soldaten beim P...,2017-02-04 12:50:25.000,Louvre nach Macheten-Angriff wieder geöffnet
4,ld.149385,Unknown,Unglück in Panama,"<div data-component=""subtitle"" class=""subtitle...",117,Panorama,Bei einem Busunglück in Panama sind 17 Persone...,2017-03-06 07:31:21.000,Bus prallt gegen eine Mauer und stürzt in Fluss


In [19]:
articles.isna().sum()

nzz_id            0
author            0
catchline         0
content           0
content_length    0
department        0
lead_text         0
pub_date          0
title             0
dtype: int64

In [20]:
articles.shape

(22019, 9)

In [21]:
# czyszczenie content
def clean_html(data):
    html_clean = re.compile(r'<.*?>')
    data = html_clean.sub(r' ',data)
    return data

def clean_data(row):
    row = str(BeautifulSoup(row))
    row = clean_html(row).replace(u'\xa0', u' ')
    return row

In [22]:
articles['content'] = articles['content'].apply(clean_data)

In [23]:
articles.content[0]

'   Obligationenfonds mit fixer Laufzeit  Es gibt noch interessante Varianten  Claudia Gabriel  Die Idee ist gut: Statt einer einzigen Obligation einen Anteil an einem diversifizierten Portfolio erwerben, Coupons bekommen und am Ende der Laufzeit das Geld zurück. Doch für Schweizer Anleger schrumpft die Auswahl.  Was tun, wenn einheimische Staatsanleihen sehr tief oder negativ rentieren, aber man als Anleger eine Geldsumme zu «parkieren» hätte, die man in einigen Jahren sicher ganz wieder zurück haben will? Das ist ein häufiges Problem privater Anleger. Der traditionelle Ausweg aus diesem Dilemma lautete, Unternehmens- oder Staatsanleihen mit etwas höheren Coupons zu kaufen. Damit handelt man sich aber leicht Klumpenrisiken ein - wie zum Beispiel um die Jahrtausendwende die Obligationäre der konkursiten Swissair oder die Gläubiger Argentiniens schmerzlich feststellen mussten. Schön wäre es also, ein solches Investment auf diversifizierte Art tätigen zu können.  Die Preise «gewöhnlicher

In [37]:
articles.loc[articles.content =='   Unknown   ', 'content'] = ''

In [38]:
articles.content[1]

''

In [39]:
articles.to_csv("./../articles_clean_new.csv", encoding="utf-8", index=False)